In [1]:
import tensorflow as tf
from net.generator import *
from utils.cosine_anealing import *
#from utils.losses import *
from tensorflow.keras.optimizers import Adam

from net.wide_resnet import WideResidualNetwork
#from train_scratch import *
import numpy as np

In [2]:
tf.__version__

'2.0.0'

In [3]:
def get_model_outputs(model, input):
    """
    given model and the input data, outputs the logits and activations required for attention training

    :param model: either student or teacher model
    :param input: input batch of images
    :param mode: 0 for test mode or 1 for train mode
    :return: [logits, activations of 3 main blocks]

    TODO:
        1. use ```get_intm_outputs_of``` instead
        2. Remove this when fully migrated
    """
    output_layer_names = ['logits', 'attention1', 'attention2', 'attention3']
    get_outputs = K.function([model.layers[0].input],
                             [model.get_layer(l).output for l in output_layer_names])

    return get_outputs([input])

In [4]:
z_dim = 100
batch_size = 128
ng_batches = 1
ns_batches = 10
attn_beta = 250
total_n_pseudo_batches = 10
n_generator_items = ng_batches + ns_batches
total_batches = 0
student_lr = 2e-3
generator_lr = 1e-3
number_of_batches = 10

teacher_model = WideResidualNetwork(16, 1, input_shape=(32, 32, 3), dropout_rate=0.0)
teacher_model.load_weights('saved_models/cifar10_WRN-16-1_model.005.h5')

student_model = WideResidualNetwork(16, 1, input_shape=(32, 32, 3), dropout_rate=0.0)
student_optimizer=Adam(learning_rate=student_lr)
student_scheduler = CosineAnnealingScheduler(T_max=number_of_batches, eta_max=student_lr, eta_min=0)
"""
compile student model with loss and lr_scheduler
"""
generator_model = generator(100)
generator_optimizer=Adam(learning_rate=generator_lr)
generator_scheduler = CosineAnnealingScheduler(T_max=number_of_batches, eta_max=generator_lr, eta_min=0)
"""
compile generator model with loss and lr_scheduler
"""

student_model.trainable = True
teacher_model.trainable = False
generator_model.trainable = True

gen_loss_metric = tf.keras.metrics.Mean()

# algo 1

    for total_batches in range(total_n_pseudo_batches):
    
        z = tf.random.normal([batch_size, z_dim])
        pseudo_images = get_gen_images(z)
        teacher_logits, *teacher_activations = get_model_outputs(teacher_model, pseudo_images, mode=0)

        #generator training
        for ng in range(ng_batches):
            student_logits, *student_activations = get_model_outputs(student_model, pseudo_images, mode=1)
            generator_loss = generator_loss(teacher_logits, student_logits)

            #################################
            # BACK PROP AND tick schedulers #
            #################################  

        for ns in range(ns_batches):
            student_logits, *student_activations = get_model_outputs(student_model, pseudo_images, mode=1)
            student_loss = student_loss(teacher_logits, teacher_activations, 
                                        student_logits, student_activations, attn_beta)

            #################################
            # BACK PROP AND tick schedulers #
            #################################   

        ######################################################
        ### Val accuracy computation and best model saving ###
        ######################################################    

In [32]:
t_model = Model(teacher_model.input, [teacher_model.get_layer(l).output for l in output_layer_names])
s_model = Model(student_model.input, [student_model.get_layer(l).output for l in output_layer_names])

for total_batches in range(total_n_pseudo_batches):
    with tf.GradientTape() as tape:
        z = tf.random.normal([batch_size, z_dim])
        pseudo_images = generator_model(z)
        #teacher_logits, *teacher_activations = get(teacher_model, pseudo_images, mode=0)
        teacher_logits, *teacher_activations = t_model(pseudo_images)

        #generator training
        for ng in range(ng_batches):
            #pseudo_images = generator_model(z)
            #student_logits = student_model(pseudo_images)
            #teacher_logits = teacher_model(pseudo_images)
            student_logits, *student_activations = s_model(pseudo_images)
            #teacher_logits, *teacher_activations = get_model_outputs(teacher_model, pseudo_images)
            #act1 = teacher_model
            # teacher_logits, *teacher_activations = get_intm_outputs_of(teacher_model, pseudo_images, "eval")
            generator_loss = kd_loss(tf.math.softmax(teacher_logits), tf.math.softmax(student_logits))

            grads = tape.gradient(generator_loss, generator_model.trainable_weights)
            generator_optimizer.apply_gradients(zip(grads, generator_model.trainable_weights))

            gen_loss_metric(generator_loss)

            if total_batches % 2 == 0:
                print('step %s: generator mean loss = %s' % (total_batches, gen_loss_metric.result()))
                
        for ns in range(2):
            student_logits, *student_activations = s_model(pseudo_images)
            student_loss = student_loss(teacher_logits, teacher_activations, 
                                    student_logits, student_activations, attn_beta)
            
            print(student_loss)
            

step 0: generator mean loss = tf.Tensor(0.9093225, shape=(), dtype=float32)


TypeError: 'tensorflow.python.framework.ops.EagerTensor' object is not callable

In [6]:
teacher_logits, *teacher_activations = get_model_outputs(teacher_model, pseudo_images)

In [7]:
teacher_logits.shape

(128, 10)

In [9]:
teacher_activations[1].shape

(128, 16, 16, 32)

In [10]:
teacher_logits2 = teacher_model(pseudo_images)

In [11]:
teacher_logits2 == teacher_logits

<tf.Tensor: id=4624, shape=(128, 10), dtype=bool, numpy=
array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])>

In [17]:
teacher_logits[1]

array([ 1.1260237 , -2.5544784 ,  3.9817953 ,  0.16220087,  1.5536513 ,
       -0.6239331 ,  0.32931316, -2.3540037 ,  1.2673128 , -3.39674   ],
      dtype=float32)

In [16]:
teacher_logits2[1]

<tf.Tensor: id=4628, shape=(10,), dtype=float32, numpy=
array([4.5154642e-02, 1.1383623e-03, 7.8514206e-01, 1.7223423e-02,
       6.9249831e-02, 7.8470409e-03, 2.0356141e-02, 1.3910593e-03,
       5.2007198e-02, 4.9033295e-04], dtype=float32)>

In [19]:
tf.math.softmax(teacher_logits[1])

<tf.Tensor: id=4635, shape=(10,), dtype=float32, numpy=
array([4.5154642e-02, 1.1383623e-03, 7.8514206e-01, 1.7223423e-02,
       6.9249831e-02, 7.8470409e-03, 2.0356141e-02, 1.3910593e-03,
       5.2007198e-02, 4.9033295e-04], dtype=float32)>

In [26]:
with tf.GradientTape() as tape:
    pseudo_images = generator_model(z)
    #student_logits = student_model(pseudo_images)
    #teacher_logits = teacher_model(pseudo_images)
    output_layer_names = ['logits', 'attention1', 'attention2', 'attention3']
    get_outputs = K.function([student_model.layers[0].input],
                             [student_model.get_layer(l).output for l in output_layer_names])
    
    student_logits, *student_activations = get_outputs(pseudo_images)
    #teacher_logits, *teacher_activations = get_model_outputs(teacher_model, pseudo_images)

AttributeError: 'NoneType' object has no attribute 'definition'

In [6]:
output_layer_names = ['logits', 'attention1', 'attention2', 'attention3']
t_model = Model(teacher_model.input, [teacher_model.get_layer(l).output for l in output_layer_names])
s_model = Model(student_model.input, [student_model.get_layer(l).output for l in output_layer_names])

In [8]:
z = tf.random.normal([batch_size, z_dim])
pseudo_images = generator_model(z)
student_logits, *student_activations = s_model(pseudo_images)
teacher_logits, *teacher_activations = t_model(pseudo_images)

In [27]:
import numpy as np
from tensorflow.keras.losses import KLD
from tensorflow.keras.utils import normalize

def kd_loss(teacher_logits, student_logits):
	return KLD(teacher_logits, student_logits)

def f_act(activations):
	temp1 = np.mean(np.power(activations,2), axis=-1)
	temp2 = temp1.reshape((temp1.shape[0],-1))
	return normalize(temp2)

def attention_loss(teacher_activations, student_activations):
	ta = f_act(teacher_activations)
	sa = f_act(student_activations)
	return np.mean(np.power(ta-sa,2))

def generator_loss(teacher_logits, student_logits):
	return kd_loss(teacher_logits, student_logits)

def student_loss(teacher_logits, teacher_activations, student_logits, student_activations, attn_beta):
	kld_loss = kd_loss(teacher_logits, student_logits)

	attn_loss = 0
	for i in range(len(teacher_activations)):
		attn_loss += attention_loss(teacher_activations[i], student_activations[i])

	total_loss = kld_loss + attn_beta*attn_loss
	return total_loss

In [28]:
student_loss(teacher_logits, teacher_activations, student_logits, student_activations, attn_beta)

<tf.Tensor: id=15359, shape=(128,), dtype=float32, numpy=
array([63.246574, 62.530354, 56.460114, 55.42478 , 63.27556 , 56.38376 ,
       60.581436, 63.82227 , 61.98921 , 56.03036 , 59.189995, 55.950417,
       62.70999 , 63.79803 , 64.08092 , 56.82087 , 56.823624, 62.2207  ,
       56.236553, 59.76841 , 61.870365, 56.21703 , 62.96882 , 62.265327,
       61.337734, 54.915745, 63.484383, 59.215508, 58.232044, 55.48815 ,
       61.401527, 67.845535, 56.594025, 64.69385 , 55.39949 , 66.48599 ,
       63.981632, 55.62022 , 59.985752, 59.208084, 57.66606 , 68.60975 ,
       75.42643 , 64.97571 , 66.93693 , 61.816204, 61.56173 , 64.500946,
       66.854095, 57.82804 , 63.552296, 63.225067, 58.97706 , 57.122128,
       59.35469 , 58.93611 , 61.65521 , 59.431847, 62.997116, 69.90731 ,
       60.716286, 63.285748, 57.051826, 58.70556 , 62.119232, 57.97173 ,
       61.722897, 55.113766, 57.090717, 60.315895, 55.701664, 58.502346,
       59.46634 , 59.06769 , 60.498367, 57.339485, 55.54771 , 61.2

In [15]:
f_act(teacher_activations)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'conjugate'

In [20]:
temp1 = np.mean(np.power(teacher_activations[1],2), axis=-1)

In [21]:
temp2 = temp1.reshape((temp1.shape[0],-1))

In [22]:
temp2.shape

(128, 256)

In [26]:
normalize(temp2)

array([[0.03883561, 0.05362019, 0.05910356, ..., 0.04889502, 0.05477028,
        0.03366701],
       [0.04036226, 0.05192825, 0.05217778, ..., 0.0457527 , 0.05595335,
        0.03459352],
       [0.03507579, 0.04746699, 0.05483653, ..., 0.04617997, 0.05300716,
        0.03272118],
       ...,
       [0.03824976, 0.05382752, 0.06030249, ..., 0.04798065, 0.05820921,
        0.0329163 ],
       [0.04094665, 0.05460351, 0.05658972, ..., 0.0450317 , 0.05541056,
        0.03339447],
       [0.03628492, 0.05067764, 0.05372652, ..., 0.04655078, 0.04968923,
        0.02940216]], dtype=float32)

In [25]:
temp2

array([[3.407847 , 4.7052016, 5.1863704, ..., 4.2905655, 4.806123 ,
        2.9542987],
       [3.4944272, 4.4957714, 4.517375 , ..., 3.961114 , 4.8442516,
        2.9949899],
       [3.002557 , 4.063267 , 4.6941147, ..., 3.953096 , 4.5375175,
        2.8009977],
       ...,
       [3.1845975, 4.4815707, 5.0206633, ..., 3.9947724, 4.8463817,
        2.7405448],
       [3.3936718, 4.525557 , 4.690174 , ..., 3.7322416, 4.5924454,
        2.767745 ],
       [3.360858 , 4.6939697, 4.97637  , ..., 4.3117237, 4.60242  ,
        2.7233477]], dtype=float32)